In [ ]:
from datetime import datetime
from pathlib import Path
import pprint
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
from pyseir.models.seir_model import SEIRModel
from pyseir.parameters.parameter_ensemble_generator import ParameterEnsembleGenerator
from pyseir.models import suppression_policies

In [ ]:
from pyseir.outcomes import OutcomesSampler, ContinuousParameter, OutcomeModels

In [ ]:
t_list = np.linspace(0, 1080, 1080)
fips_list = ["06037", "36061"]

parameter_space = [ContinuousParameter('R0', 2, 4, alpha=0.8, beta=1), 
                   ContinuousParameter('delta', 1.0 / 30, 1, alpha=0.5, beta=1), 
                   ContinuousParameter('suppression_policy', 0.01, 1, alpha=1.25, beta=1.25)]

outcome_fs = {"R-final": lambda x: np.log10(x[-1]),
              "D-final": lambda x: np.log10(x[-1]),
              "HGen-max": lambda x: np.log10(np.max(x)), 
              "HGen-time_to_max": lambda x: np.log10(np.argmax(x)),  
              "HICU-max": lambda x: np.log10(np.max(x)), 
              "HICU-time_to_max": lambda x: np.log10(np.argmax(x)),
              "HVent-max": lambda x: np.log10(np.max(x)), 
              "HVent-time_to_max": lambda x: np.log10(np.argmax(x)),
              }

num_outcomes = len(outcome_fs)

def fn_approximator(oc, X, y):
    kernel = RationalQuadratic(length_scale=0.3)
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=0, n_restarts_optimizer=10)
    gpr.fit(X, y)
    return (oc, gpr)

In [ ]:
%%time
num_samples = 100 # number of samples from parameter space
fips_models = {}
for fips in fips_list:
    parameter_generator = ParameterEnsembleGenerator(fips=fips,
                                                    N_samples=1000,
                                                    t_list=t_list)
    os = OutcomesSampler(parameter_generator, parameter_space, outcome_fs, num_samples=num_samples)
    model_base = Path.cwd() / Path("models/sensitivity_analysis")
    ts_path = model_base / Path(fips) / Path("{:%Y%m%d-%H%M}".format(datetime.now()))
    ts_path.mkdir(parents=True, exist_ok=True)
    samples_path = Path("outcome_samples.pkl")
    joblib.dump(os, ts_path / samples_path, compress=True)
    parameter_names = [p.parameter_name for p in parameter_space]
    om = OutcomeModels(os.outcomes_df, parameter_names, fn_approximator)
    models_path = Path("outcome_models.pkl")
    joblib.dump(om.outcome_models, ts_path / models_path, compress=9)
    fips_models[fips] = om.outcome_models

In [ ]:
def plot_outcomes(model, grid, outcome_name, num_outcomes, idx):
    plt.subplot(1, num_outcomes, idx)
    x_ = grid["suppression_policy"]
    y_mean, y_cov = model.predict(grid, return_cov=True)
    _ = plt.plot(x_, y_mean)
    _ = plt.fill_between(x_, y_mean - np.sqrt(np.diag(y_cov)),
                        y_mean + np.sqrt(np.diag(y_cov)),
                        alpha=0.5, color='k')
    _ = plt.xlabel("Suppression Policy")
    _ = plt.ylabel(outcome_name)

In [ ]:
grid = pd.DataFrame({"R0": 2.5 * np.ones(100), 
                     "delta": 1/14 * np.ones(100), 
                     "suppression_policy": np.linspace(0.01, 1, 100)})

for fips, model in fips_models.items():   
    fig = plt.figure(figsize=(40, 10))
    fig.suptitle(f"Outcome responses for {fips}")
    for i, outcome_name in enumerate(model.keys()): 
        plot_outcomes(model[outcome_name], grid, outcome_name, num_outcomes, i+1)
 
